In [2]:
import pandas as pd
from conexao import get_engine

engine = get_engine()

# Ler tabelas
clientes = pd.read_sql("SELECT * FROM etl_course.clientes", engine)
pedidos = pd.read_sql("SELECT * FROM etl_course.pedidos", engine)

print(clientes.head())
print(pedidos.head())

   id            nome                     email          cidade
0   1       Ana Souza       ana.souza@email.com       São Paulo
1   2      Bruno Lima      bruno.lima@email.com  Rio de Janeiro
2   3    Carla Mendes    carla.mendes@email.com  Belo Horizonte
3   4  Diego Ferreira  diego.ferreira@email.com    Porto Alegre
4   5   Elisa Martins   elisa.martins@email.com        Curitiba
   id  cliente_id  valor data_pedido
0   1           1  250.0  2023-01-10
1   2           1  320.0  2023-03-15
2   3           2  150.0  2023-02-05
3   4           2  180.0  2023-04-12
4   5           2  210.0  2023-06-20


Valor total por cliente

In [5]:
valor_total = pedidos.groupby("cliente_id")["valor"].sum().reset_index()
print(valor_total)
df = clientes.merge(valor_total, left_on="id", right_on="cliente_id", how="left")
print(df)
print(df[["nome", "valor"]])

   cliente_id   valor
0           1   570.0
1           2   540.0
2           3  1550.0
3           4   220.0
4           5   900.0
   id            nome                     email          cidade  cliente_id  \
0   1       Ana Souza       ana.souza@email.com       São Paulo           1   
1   2      Bruno Lima      bruno.lima@email.com  Rio de Janeiro           2   
2   3    Carla Mendes    carla.mendes@email.com  Belo Horizonte           3   
3   4  Diego Ferreira  diego.ferreira@email.com    Porto Alegre           4   
4   5   Elisa Martins   elisa.martins@email.com        Curitiba           5   

    valor  
0   570.0  
1   540.0  
2  1550.0  
3   220.0  
4   900.0  
             nome   valor
0       Ana Souza   570.0
1      Bruno Lima   540.0
2    Carla Mendes  1550.0
3  Diego Ferreira   220.0
4   Elisa Martins   900.0


Primeiro pedido por cliente

In [6]:
pedidos["primeiro_pedido"] = pedidos.groupby("cliente_id")["data_pedido"].transform("min")
print(pedidos)

    id  cliente_id  valor data_pedido primeiro_pedido
0    1           1  250.0  2023-01-10      2023-01-10
1    2           1  320.0  2023-03-15      2023-01-10
2    3           2  150.0  2023-02-05      2023-02-05
3    4           2  180.0  2023-04-12      2023-02-05
4    5           2  210.0  2023-06-20      2023-02-05
5    6           3  500.0  2023-01-25      2023-01-25
6    7           3  700.0  2023-02-28      2023-01-25
7    8           3  350.0  2023-05-10      2023-01-25
8    9           4  100.0  2023-03-02      2023-03-02
9   10           4  120.0  2023-05-18      2023-03-02
10  11           5  900.0  2023-02-14      2023-02-14


Diferença em dias entre pedidos

In [7]:
pedidos["data_pedido"] = pd.to_datetime(pedidos["data_pedido"])
pedidos["dias_diff"] = pedidos.groupby("cliente_id")["data_pedido"].diff().dt.days
print(pedidos)

    id  cliente_id  valor data_pedido primeiro_pedido  dias_diff
0    1           1  250.0  2023-01-10      2023-01-10        NaN
1    2           1  320.0  2023-03-15      2023-01-10       64.0
2    3           2  150.0  2023-02-05      2023-02-05        NaN
3    4           2  180.0  2023-04-12      2023-02-05       66.0
4    5           2  210.0  2023-06-20      2023-02-05       69.0
5    6           3  500.0  2023-01-25      2023-01-25        NaN
6    7           3  700.0  2023-02-28      2023-01-25       34.0
7    8           3  350.0  2023-05-10      2023-01-25       71.0
8    9           4  100.0  2023-03-02      2023-03-02        NaN
9   10           4  120.0  2023-05-18      2023-03-02       77.0
10  11           5  900.0  2023-02-14      2023-02-14        NaN


E-mails duplicados

In [7]:
print(clientes["email"].duplicated().sum())
print(clientes["email"].duplicated(keep=False))

0
0    False
1    False
2    False
3    False
4    False
Name: email, dtype: bool


Pedidos com Clientes inexistentes


In [8]:
pedidos_invalidos = pedidos[~pedidos["cliente_id"].isin(clientes["id"])]
print(pedidos_invalidos)

Empty DataFrame
Columns: [id, cliente_id, valor, data_pedido]
Index: []
